# Example Notebook to work with SMA data

In [1]:
from sma import Config, Report, ReportIO

config = Config.from_file("examples/minimal.yml")


reports = ReportIO.load_from_config(config)

/Users/b/.local/share/virtualenvs/sustainability-measurments-WjoA9q4p/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
reports[0].environment.nodes


[]

In [4]:
from sma import SMARun, Config
from datetime import datetime, timedelta
config = Config.from_file("examples/minimal.yml")
foo = config.create_environment_observation()
bar = foo.observe_environment(SMARun(datetime.now(),datetime.now(),datetime.now(),datetime.now(),"foo",None))

bar.nodes

[Node(name='ise-kmaster', labels={'__name__': 'kube_node_info', 'app_kubernetes_io_component': 'metrics', 'app_kubernetes_io_managed_by': 'Helm', 'app_kubernetes_io_name': 'kube-state-metrics', 'app_kubernetes_io_part_of': 'kube-state-metrics', 'container_runtime_version': 'containerd://1.6.20', 'internal_ip': '130.149.158.130', 'job': 'kubernetes-service-endpoints', 'kernel_version': '6.1.0-40-cloud-amd64', 'kubelet_version': 'v1.29.0', 'namespace': 'monitoring', 'os_image': 'Debian GNU/Linux 12 (bookworm)', 'pod_cidr': '192.168.0.0/24', 'system_uuid': 'b8e493ca-feaf-4525-bc61-f6e3c424f5fc', 'node_info': 1}),
 Node(name='ise-knode1', labels={'__name__': 'kube_node_info', 'app_kubernetes_io_component': 'metrics', 'app_kubernetes_io_managed_by': 'Helm', 'app_kubernetes_io_name': 'kube-state-metrics', 'app_kubernetes_io_part_of': 'kube-state-metrics', 'container_runtime_version': 'containerd://1.6.20', 'internal_ip': '130.149.158.131', 'job': 'kubernetes-service-endpoints', 'kernel_versi

In [45]:
useless_tags=['layer', 'unit', 'app_kubernetes_io_instance', 'app_kubernetes_io_version','helm_sh_chart', 'instance', 'kubeproxy_version', 'service','__name__','app_kubernetes_io_managed_by','app_kubernetes_io_part_of', 'app_kubernetes_io_name','app_kubernetes_io_component','job']
from sma import Pod
from datetime import datetime, timedelta
pods = foo.queries['container_infos'].observe(datetime.now()-timedelta(1),datetime.now())

pods = pods.drop(columns=useless_tags+['timestamp'], errors='ignore').reset_index()
# pods.groupby(['pod','container','node'])['timestamp'].agg(['min','max','count'])
pods = pods.groupby([ 'node','pod','container', 'container_id', 'image_id',  'namespace',   'uid'])['timestamp'].agg(['min', 'max', 'count']).reset_index()

In [46]:
pods

,node,pod,container,container_id,image_id,namespace,uid,min,max,count
0,ise-knode1,45824453932d2cbe15273c34cb5fd2769eab1a1558cbe7...,extract,containerd://277ad7e2aa61efef0c61c27627579d225...,quay.io/operator-framework/opm@sha256:d4584b8e...,olm,d2cdd6f5-159e-4cb8-84f7-720a3253dd84,2025-12-18 11:14:11.085999966+00:00,2025-12-18 13:43:11.085999966+00:00,300
1,ise-knode1,activator-86788466b5-9589d,activator,containerd://28c79d259df0f7796334323522cc20aea...,gcr.io/knative-releases/knative.dev/serving/cm...,knative-serving,4e5425fd-7616-406d-9906-4bd935750581,2025-12-18 13:41:11.085999966+00:00,2025-12-18 13:43:11.085999966+00:00,6
2,ise-knode1,activator-86788466b5-9589d,activator,containerd://29c1dd09d4a73f18e9c1428c564ec9d97...,gcr.io/knative-releases/knative.dev/serving/cm...,knative-serving,4e5425fd-7616-406d-9906-4bd935750581,2025-12-18 11:39:11.085999966+00:00,2025-12-18 11:41:11.085999966+00:00,6
3,ise-knode1,activator-86788466b5-9589d,activator,containerd://2cb4be46cb9cc6079ff47160e3504c09b...,gcr.io/knative-releases/knative.dev/serving/cm...,knative-serving,4e5425fd-7616-406d-9906-4bd935750581,2025-12-18 13:33:11.085999966+00:00,2025-12-18 13:40:11.085999966+00:00,16
4,ise-knode1,activator-86788466b5-9589d,activator,containerd://32e5a433bc811036e0088cca8e73f52fc...,gcr.io/knative-releases/knative.dev/serving/cm...,knative-serving,4e5425fd-7616-406d-9906-4bd935750581,2025-12-18 12:37:11.085999966+00:00,2025-12-18 12:44:11.085999966+00:00,16
...,...,...,...,...,...,...,...,...,...,...
251,ise-knode1,synthetic-generator-6f856fd5dd-82dxr,synthetic-generator,containerd://73532f391a476fa25b2506f4abd89f283...,docker.io/sepidmas/reuse_scale-synthetic-gener...,pipeline,1016bcf2-2633-4be7-9219-bdc70ce83219,2025-12-18 11:14:11.085999966+00:00,2025-12-18 13:43:11.085999966+00:00,300
252,ise-knode1,teadal-gitlab-gitlab-runner-b9fcb8cf8-d2xz6,teadal-gitlab-gitlab-runner,containerd://ef4ce44585813a7bf726f446e87003f6d...,registry.gitlab.com/gitlab-org/gitlab-runner@s...,teadal-tools,02eebf73-afdd-48d7-a297-7e34e2366c5a,2025-12-18 11:14:11.085999966+00:00,2025-12-18 13:43:11.085999966+00:00,300
253,ise-knode1,webhook-558b975745-q7kkk,webhook,containerd://11750e3a85bfaf86c2ebe4fe70cf939e4...,gcr.io/knative-releases/knative.dev/serving/cm...,knative-serving,a0f5f5a5-8328-4756-b5fd-3bb1cb66ade6,2025-12-18 11:14:11.085999966+00:00,2025-12-18 13:43:11.085999966+00:00,300
254,ise-knode1,workflow-controller-55cf7c9df5-lsmn2,workflow-controller,containerd://b66507ff4180ee83bc8701f2308025eb3...,quay.io/argoproj/workflow-controller@sha256:33...,argo,698fa79c-89f1-46a7-8275-4cfc2f9e8558,2025-12-18 11:14:11.085999966+00:00,2025-12-18 13:43:11.085999966+00:00,300
